## PyTorch implementation of Deep Bilteral Learning for Real Time Image Enhancement

In [15]:
from __future__ import division, print_function, unicode_literals
import numpy as np
from PIL import Image
import os, sys, glob
# import matplotlib.pyplot as plt
#Torch Imports
import torch
import torchvision
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as data
import torchvision.models as models

In [21]:
size = (256, 256)
batch_size = 100
learning_rate = 0.01
num_epoch = 5

In [22]:
class Dataset(data.Dataset):
    def __init__(self, root_dir, train, transform=None):
        #Init Function
        super(Dataset, self).__init__()
        self.root_dir = root_dir
        self.train = train
        self.transform = transform
        
        self.full_res = []
        self.low_res = []
        
        if (train):
            dir = self.root_dir + '/train/'
        else :
            dir = self.root_dir + '/test/'
        
        for img_path in glob.glob (dir + '*.jpg'):
            
            himage = Image.open (img_path)
            limage = himage.resize (size)
            
            self.full_res.append (himage)
            self.low_res.append (limage)

    def __len__(self):
        #Length function ?
        return length (self.full_res)

    def __getitem__(self, idx):
        #Accessor Function
        return (self.full_res[idx], self.low_res[idx])

In [23]:
train_dataset = Dataset (root_dir = '../data', train = True)
im, im2 = train_dataset.__getitem__(0)
im2.show()

In [ ]:
class LocalFeatureNet (nn.Module):

    def fusionLayer (self, localfeat, globalfeat, bias = True):
        self.local_feature = localfeat
        self.global_feature = globalfeat
            
        self.weight = nn.Parameter(torch.Tensor(output_features, input_features))
        if bias:
            self.bias = nn.Parameter(torch.Tensor(output_features))
        else:
            # You should always register all possible parameters, but the
            # optional ones can be None if you want.
            self.register_parameter('bias', None)

        # Not a very smart way to initialize weights
        self.weight.data.uniform_(-0.1, 0.1)
        if bias is not None:
            self.bias.data.uniform_(-0.1, 0.1)
        
    def bilateralGrid (self, locaFeat, globalFeat, bias = True):
        
    
    def __init__(self):
        super (LocalFeatureNet, self).__init__()
        
        self.relu  = nn.ReLU (inplace = True)
        
        self.conv1 = nn.Conv2d (in_channels = 3,   out_channels = 8, kernel = 3, stride = 2)
        self.conv2 = nn.Conv2d (in_channels = 8,  out_channels = 16, kernel = 3, stride = 2)
        self.conv3 = nn.Conv2d (in_channels = 16, out_channels = 32, kernel = 3, stride = 2)
        self.conv4 = nn.Conv2d (in_channels = 32, out_channels = 64, kernel = 3, stride = 2)
        
        
        self.localconv1 = nn.Conv2d (in_channels = 64, out_channels = 64, kernel = 3, stride = 1)
        self.localconv2 = nn.Conv2d (in_channels = 64, out_channels = 64, kernel = 3, stride = 1)
        
        self.globalconv1 = nn.Conv2d (in_channels = 64, out_channels = 64, kernel = 3, stride = 2)
        self.globalconv2 = nn.Conv2d (in_channels = 64, out_channels = 64, kernel = 3, stride = 2)
        
        self.globalfc1 = nn.Linear (256, 256)
        self.globalfc2 = nn.Linear (256, 128)
        self.globalfc3 = nn.Linear (128, 64)
        
    def forward (self, x):
        
        x = self.relu (self.conv1 (x))
        x = self.relu (self.conv2 (x))
        x = self.relu (self.conv3 (x))
        x = self.relu (self.conv4 (x))
        
        print x.size()
        
        y = self.localconv1 (x)
        y = self.localconv2 (y)
        
        print y.size()
        
        z = self.globalconv1 (x)
        z = self.globalconv2 (z)
        z = self.globalfc1 (z)
        z = self.globalfc2 (z)
        z = self.globalfc3 (z)
        
        print z.size()
        
        fused = self.fusionLayer (y, z)
        
        print fused.size()
        
        bilat = self.bilateralGrid (fused)
        
        print bilat.size()
        
        return bilat